In [ ]:
from modelClasses import BasicModel
from analyzerClasses import BasicAnalyzer
import tableGetter
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import numpy as np
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, RepeatVector, TimeDistributed
import tensorflow as tf
from time import sleep
from datetime import datetime, date, timedelta, time
from decimal import *

df = tableGetter.spyHistory()

tradeData = pd.DataFrame(columns=['Model','Analyzer','Action','Date','Time','Price']) 
totalData = pd.DataFrame(columns=['Model','Analyzer','AverageWin']) 

new_day_data = df.loc[df['date'] == (datetime.today() - timedelta(days=2)).date()]

df = df.loc[df['date'] != (datetime.today() - timedelta(days=2)).date()]

currentModel = BasicModel(df)
new_day_data[currentModel.inputs] = currentModel.inputScalar.transform(new_day_data[currentModel.inputs])

currentModel.build()

currentAnalyzer = BasicAnalyzer()

for i in range(1,len(new_day_data)-1): 
    todaysData = pd.DataFrame(new_day_data[:i])

    currentTick = todaysData.iloc[-1]
    todaysData = todaysData.loc[todaysData['time'].isin(todaysData.time.unique())]
    loadedModel = tf.keras.models.load_model('model_'+str(todaysData.iloc[-1]['time']).replace(':','_')+'.keras')
                
    X = np.array(todaysData[currentModel.inputs])
    input_sequence = X.reshape(1, len(todaysData), currentModel.numInput)
    predicted_sequence = currentModel.outputScalar.inverse_transform(pd.DataFrame(loadedModel.predict(input_sequence)[0], columns=['Difference']))

    todaysData[['Difference']] = currentModel.outputScalar.inverse_transform(todaysData[['Difference']])

    actionTaken = currentAnalyzer.action(predicted_sequence,todaysData)

    if actionTaken != '':
        tradeData.loc[len(tradeData.index)] = [currentAnalyzer.name, currentModel.name, actionTaken, str(todaysData['Datet'].values[0]), str(todaysData.iloc[-1]['time']).replace(':','_'), todaysData.iloc[-1]['Close']] 

totalWin =  currentAnalyzer.totalWin + abs(currentAnalyzer.lastPrice  - todaysData.iloc[-1]['Close'])
totalData.loc[len(totalData.index)] = [currentAnalyzer.name, currentModel.name, totalWin] 

In [ ]:
from modelClasses import BasicModel, BatchClipEarlyStopD, BatchClipEarlyStopValLossD
from analyzerClasses import BasicAnalyzer, CrossoverAnalyzer, CrossoverAnalyzerUpdated
import tableGetter
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import numpy as np
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, RepeatVector, TimeDistributed
import tensorflow as tf
from time import sleep
from datetime import datetime, date, timedelta, time
from decimal import *


df = tableGetter.SpyHistory()

tradeData = tableGetter.TradeData()
totalData = tableGetter.AverageWinning()

testingData = df.loc[df['date'] > (datetime.today() - timedelta(days=5)).date()]

absoluteTotal = 0
   
for currentDate in testingData.date.unique():

    trainingData = df.loc[df['date'] < currentDate]
    currentModel = BatchClipEarlyStopValLossD(trainingData)

    currentDaysData = df.loc[df['date']==currentDate]
    currentDaysData[currentModel.inputs] = currentModel.inputScalar.transform(currentDaysData[currentModel.inputs])

    currentModel.build()

    currentAnalyzer = BasicAnalyzer()
    currentAnalyzer.tradeNum = tradeData.TradeNumber.unique()[-1] + 1
    
    for i in range(1,len(currentDaysData)-1): 
        todaysData = pd.DataFrame(currentDaysData[:i])

        currentTick = todaysData.iloc[-1]
        todaysData = todaysData.loc[todaysData['time'].isin(todaysData.time.unique())]
        loadedModel = tf.keras.models.load_model('model_'+str(todaysData.iloc[-1]['time']).replace(':','_')+'.keras')
                    
        X = np.array(todaysData[currentModel.inputs])
        input_sequence = X.reshape(1, len(todaysData), currentModel.numInput)
        predicted_sequence = currentModel.outputScalar.inverse_transform(pd.DataFrame(loadedModel.predict(input_sequence)[0], columns=['Difference']))

        todaysData[['Difference']] = currentModel.outputScalar.inverse_transform(todaysData[['Difference']])

        actionTaken = currentAnalyzer.action(predicted_sequence,todaysData)

        if actionTaken != '':
            if currentAnalyzer.closed:
                tradeData.loc[len(tradeData.index)] = [currentAnalyzer.name, currentModel.name, actionTaken, str(todaysData['date'].values[0]), str(todaysData.iloc[-1]['time']).replace(':','_'), todaysData.iloc[-1]['Close'], currentAnalyzer.tradeNum, currentAnalyzer.profit] 
                tradeData['Profit'].loc[tradeData['TradeNumber'] == currentAnalyzer.tradeNum] = currentAnalyzer.profit
                currentAnalyzer.tradeNum = currentAnalyzer.tradeNum + 1
            if actionTaken == 'GOLONG' or actionTaken == 'GOSHORT':
                tradeData.loc[len(tradeData.index)] = [currentAnalyzer.name, currentModel.name, actionTaken, str(todaysData['date'].values[0]), str(todaysData.iloc[-1]['time']).replace(':','_'), todaysData.iloc[-1]['Close'], currentAnalyzer.tradeNum, 0] 

    if(currentAnalyzer.long):
        profit = todaysData.iloc[-1]['Close'] - currentAnalyzer.lastPrice
        currentAnalyzer.totalWin = currentAnalyzer.totalWin + profit
        tradeData.loc[len(tradeData.index)] = [currentAnalyzer.name, currentModel.name, 'SELL', str(todaysData['date'].values[0]), str(todaysData.iloc[-1]['time']).replace(':','_'), todaysData.iloc[-1]['Close'], currentAnalyzer.tradeNum, profit] 
        tradeData['Profit'].loc[tradeData['TradeNumber'] == currentAnalyzer.tradeNum] = profit
        currentAnalyzer.tradeNum = currentAnalyzer.tradeNum + 1
    elif(currentAnalyzer.short):
        profit = currentAnalyzer.lastPrice - todaysData.iloc[-1]['Close']
        currentAnalyzer.totalWin = currentAnalyzer.totalWin + profit
        tradeData.loc[len(tradeData.index)] = [currentAnalyzer.name, currentModel.name, 'BUY', str(todaysData['date'].values[0]), str(todaysData.iloc[-1]['time']).replace(':','_'), todaysData.iloc[-1]['Close'], currentAnalyzer.tradeNum, profit] 
        tradeData['Profit'].loc[tradeData['TradeNumber'] == currentAnalyzer.tradeNum] = profit
        currentAnalyzer.tradeNum = currentAnalyzer.tradeNum + 1

    absoluteTotal = absoluteTotal + currentAnalyzer.totalWin

winningAverage = absoluteTotal / len(testingData.date.unique())
totalData.loc[len(totalData.index)] = [currentAnalyzer.name, currentModel.name, winningAverage,'9:30:00','16:00:00'] 
totalData.to_csv('averageWin.csv')  
tradeData.to_csv('tradeData.csv')  


In [ ]:
from modelClasses import BasicModel, BatchClipEarlyStopD, BatchClipEarlyStopValLossD
from analyzerClasses import BasicAnalyzer, CrossoverAnalyzer, CrossoverAnalyzerUpdated
import tableGetter
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import numpy as np
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, RepeatVector, TimeDistributed
import tensorflow as tf
from time import sleep
from datetime import datetime, date, timedelta, time
from decimal import *


df = tableGetter.SpyHistory()
currentModel = BasicModel(df)
currentModel.build()

In [ ]:
from modelClasses import BasicModel, BatchClipEarlyStopD
from analyzerClasses import BasicAnalyzer, CrossoverAnalyzer, CrossoverAnalyzerUpdated
import tableGetter
import ibapi
from ibapi.client import EClient
from ibapi.wrapper import EWrapper
from ibapi.contract import Contract
from ibapi.order import *
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import numpy as np
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, RepeatVector, TimeDistributed
import tensorflow as tf
from time import sleep
from datetime import datetime, date, timedelta, time
from sklearn.preprocessing import MinMaxScaler
import threading
from decimal import *

df = tableGetter.SpyHistory()
currentModel = BasicModel(df)
currentAnalyzer = BasicAnalyzer()

long = False
short = False
lastPrice = 0
totalWin = 0

class IBapi(EWrapper, EClient):
    def __init__(self):
        EClient.__init__(self, self)
        self.data = []
        self.df=None

    def historicalData(self, reqId, bar):
        self.data.append([bar.date, bar.open, bar.close])

    def historicalDataUpdate(self, reqId, bar):
        self.data.append([bar.date, bar.open, bar.close])
        
    def historicalDataEnd(self, reqId: int, start: str, end: str):
        print("HistoricalDataEnd. ReqId:", reqId, "from", start, "to", end)
        self.df = pd.DataFrame(self.data, columns=['DateTime', 'Open', 'Close'])
        try:
            self.df['DateTime'] = pd.to_datetime(self.df['DateTime'],unit='s').dt.tz_localize(tz='UTC')
            self.df['DateTime'] = self.df['DateTime'].dt.tz_convert(tz='US/Eastern')
            self.df['Open'] = self.df['Open'].astype(float)
            self.df['Close'] = self.df['Close'].astype(float)
            self.df['date'] = [d.date() for d in self.df['DateTime']]
            self.df['time'] = [d.time() for d in self.df['DateTime']]
            self.df['Difference'] = self.df['Close'] - self.df['Open']

        except:
            print('error loading data')
        self.df.set_index('DateTime', inplace=True)

    def nextValidId(self, orderId: int):
        super().nextValidId(orderId)
        self.nextorderId = orderId
        print('The next valid order id is: ', self.nextorderId)

    def orderStatus(self, orderId, status, filled, remaining, avgFullPrice, permId, parentId, lastFillPrice, clientId, whyHeld, mktCapPrice):
        print('orderStatus - orderid:', orderId, 'status:', status, 'filled', filled, 'remaining', remaining, 'lastFillPrice', lastFillPrice)
    
    def openOrder(self, orderId, contract, order, orderState):
        print('openOrder id:', orderId, contract.symbol, contract.secType, '@', contract.exchange, ':', order.action, order.orderType, order.totalQuantity, orderState.status)

    def execDetails(self, reqId, contract, execution):
        print('Order Executed: ', reqId, contract.symbol, contract.secType, contract.currency, execution.execId, execution.orderId, execution.shares, execution.lastLiquidity)
        
    def position(self, account: str, contract: Contract, position: Decimal, avgCost: float):
        super().position(account, contract, position, avgCost)
        print("Position.", "Account:", account, "Symbol:", contract.symbol, "SecType:", contract.secType, "Currency:", contract.currency,"Position:", decimalMaxString(position), "Avg cost:", floatMaxString(avgCost))
        
def run_loop():
    app.run()

app = IBapi()
app.connect('127.0.0.1', 7496, 126)

#Start the socket in a thread
api_thread = threading.Thread(target=run_loop, daemon=True)
api_thread.start()


#Create contract object
contract = Contract()
contract.symbol = 'SPY'
contract.secType = 'STK'
contract.exchange = 'SMART'
contract.currency = 'USD'
app.reqHistoricalData(1, contract, '', '1 D', '5 mins', 'TRADES', 0, 2, True, [])